In [1]:
%reload_ext autoreload
%autoreload 2
from hw5_funct import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from numba import jit, autojit
import multiprocessing
from itertools import repeat

X_data, Y_data, activities = get_data('./data/')

In [2]:
@autojit
def get_accuracy_basic(window, K, X_data, Y_data):
    rfscores = []
    conf_mats = np.zeros((14, 14))
    for i in range(10):
        X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, stratify=Y_data)
        vq_dict = get_vq_dict(X_train, window, K)
        X_train_VQ = vec_quantize_signals(X_train, vq_dict)
        X_test_VQ = vec_quantize_signals(X_test, vq_dict)
        rf = RandomForestClassifier(n_estimators=100, max_depth=50)
        rf.fit(X_train_VQ, Y_train)
        rfscore = rf.score(X_test_VQ,Y_test)
        rfscores.append(rfscore)
        Y_pred = rf.predict(X_test_VQ)
        conf_mats = np.add(conf_mats, confusion_matrix(Y_test,Y_pred))
    return np.mean(rfscores), conf_mats

In [3]:
window = 3*9
kvs = [400]
for K in kvs:
    rfscore, conf_mats = get_accuracy_basic(window, K, X_data, Y_data)
    print('%d, %d, %.5f' % (window, K, rfscore))
    print(conf_mats)

27, 400, 0.79762
[[ 20.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 162.   0.   3.   8.   0.   0.   0.   0.   4.   2.   2.   1.  20.]
 [  0.   0.  56.   0.   4.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  15.   0.  63.   0.   0.   0.   0.   0.   0.   0.   2.   0.   0.]
 [  0.   0.   0.   0. 194.   0.   0.   0.   0.   5.   0.   0.   1.   0.]
 [  0.   0.   0.   0.   1.   9.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.  10.   0.   0.   0.   0.]
 [  0.   1.   0.   0.  10.   0.   0. 149.   0.   9.   7.  23.   0.   1.]
 [  0.   0.   2.   0.   4.   0.   0.  29.   0.   4.  18.   2.   0.   1.]
 [  0.   0.   0.   0.   4.   0.   0.   0.   0. 192.   3.   1.   0.   0.]
 [  0.   0.   0.   0.   1.   0.   0.   5.   0.   5. 150.  39.   0.   0.]
 [  0.   1.   0.   0.   0.   0.   0.   8.   0.   0.  47. 152.   0.   0.]
 [  0.   0.   1.   0.   7.   0.   0.   1.   0.   3.   5.   0.  13.   0.]
 [  0.  11.   0.   0.   0.   0.   

In [4]:
print(activities)

['Brush_teeth', 'Climb_stairs', 'Comb_hair', 'Descend_stairs', 'Drink_glass', 'Eat_meat', 'Eat_soup', 'Getup_bed', 'Liedown_bed', 'Pour_water', 'Sitdown_chair', 'Standup_chair', 'Use_telephone', 'Walk']
